In [ ]:
# ✅ Install all required libraries for Gemini + LangChain + Google Sheets
!pip install \
    google-generativeai \
    langchain-google-genai \
    langchain_experimental \
    gspread \
    python-dotenv \
    oauth2client


INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
  Attempting uninstall: langchain-google-genai
    Found existing installation: langchain-google-genai 2.1.5
    Uninstalling lan

In [ ]:
GOOGLE_API_KEY= "xxxxxxxxxxxxxxx" #please put your correct API key


In [1]:
# ----------------------------------------
#This code snippet sets up access to Google's Gemini Pro API by securely loading an API key from a .env file
# ----------------------------------------
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

import google.generativeai as gemini
gemini.api_key = os.getenv("GOOGLE_API_KEY")


ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# Initialize a Gemini 2.0 Flash model using LangChain's integration

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)


In [ ]:
# ----------------------------------------
# 2. Import Libraries
# ----------------------------------------
import os
import gspread
import pandas as pd
from dotenv import load_dotenv
from oauth2client.service_account import ServiceAccountCredentials

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

# ----------------------------------------
# 3. Load Environment Variables
# ----------------------------------------
load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")

# ----------------------------------------
# 4. Connect to Google Sheets and Load Data
# ----------------------------------------
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/xxxxxxxxxxxxxxxxxxxxxx", scope) #Create a project in GCP and download the key
client = gspread.authorize(creds)
sheet = client.open_by_key("xxxxxxxxxxxxxxxx").sheet1 #This is the long string between /d/ and /edit in the URL for google sheets
data = pd.DataFrame(sheet.get_all_records())

# ----------------------------------------
# 6. Define Custom Tool to View Data
# ----------------------------------------
def view_data(prompt: str) -> str:
    if "show" in prompt.lower():
        return data.head().to_string()
    elif "summary" in prompt.lower():
        return data.describe().to_string()
    else:
        return "Try asking to 'show' or 'summarize' the data."

# ----------------------------------------
# 7. Create LangChain Agent for DataFrame
# ----------------------------------------
pandas_agent = create_pandas_dataframe_agent(
    llm=llm,
    df=data,
    verbose=True,
    agent_type="openai-tools",  # Gemini works with this type
    allow_dangerous_code=True
)

In [ ]:
response = pandas_agent.run("The dataframe `df` has a column called `date` (datetime), a column called `type`, and a column called `amount`.Filter the dataframe to include:- rows where the `date` is between January 1st 2025 and March 31st 2025- and the `type` is equal to cost.Then calculate the **total** of the `amount` column for these rows.Store the result in a variable called `result` and print it")
print(response)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\n\ndf['Date'] = pd.to_datetime(df['Date'])\nstart_date = pd.to_datetime('2025-01-01')\nend_date = pd.to_datetime('2025-03-31')\n\nfiltered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date) & (df['Type'] == 'Cost')]\n\nresult = filtered_df['Amount'].sum()\n\nprint(result)"}`


15332800
The total amount for the filtered rows is 15332800.```

> Finished chain.
The total amount for the filtered rows is 15332800.```
